In [1]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 14.3 MB/s eta 0:00:00


In [2]:
import nltk
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt 
import seaborn as sb
from nltk.corpus import stopwords
import warnings 
warnings.filterwarnings("ignore")
import unidecode
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
import matplotlib.animation as animation
import operator
import plotly.express as px
from collections import Counter
%matplotlib inline

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv('/content/drive/MyDrive/HATE_SPEECH_DETECTION_PROJECT/labelled_DATA_compiled/l_DATA.csv')

In [4]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"` ::My dear Heresiarchess, - Me being the ``m...",0,2
2,` :Warriorboy85 has clarified that he did not...,0,3
3,` == English == Is ``shootdown`` a proper wo...,0,4
4,`} | width=``20%`` valign=``top`` | Archives ...,0,5
...,...,...,...
94263,", like nonconvex uniform polyhedron",0,94264
94264,`Niggers and their liberal friends steal every...,1,94265
94265,Not into dressin' to impress these bitches or ...,1,94266
94266,:Replacement ref - 1UP archive not working? ...,0,94267


In [5]:
df.drop_duplicates(subset=['Text'], inplace=True)

In [6]:
num_nan_rows = df.isna().any(axis=1).sum()
print(num_nan_rows)

0


In [7]:
len(df[df['label'] == 0])

47098

In [8]:
len(df[df['label'] == 1])

47170

In [9]:
# Define function to remove backticks if they appear at the beginning and end of a string
def remove_backticks(text):
    # Check if the string starts and ends with backticks
    if text.startswith('`') or text.endswith('`'):
        # Replace double backticks with double quotes and remove the surrounding backticks
        return text[1:-1].replace('``', '"')
    # Check if the string consists of only double backticks
    elif text == '``':
        # Replace with double quotes
        return '"'
    else:
        return text

# Apply the function to the text column
df['Text'] = df['Text'].apply(remove_backticks)

In [10]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"::My dear Heresiarchess, - Me being the ""mai...",0,2
2,:Warriorboy85 has clarified that he did not ...,0,3
3,"== English == Is ""shootdown"" a proper word?...",0,4
4,"} | width=""20%"" valign=""top"" | Archives __T...",0,5
...,...,...,...
94263,", like nonconvex uniform polyhedron",0,94264
94264,Niggers and their liberal friends steal everyt...,1,94265
94265,Not into dressin' to impress these bitches or ...,1,94266
94266,:Replacement ref - 1UP archive not working? ...,0,94267


In [11]:
# Function to remove URLs from a string

url_pattern = re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')# Define URL regex pattern

def remove_urls(text):
    return url_pattern.sub('"URL HERE"', text)

# Apply function to 'text' column
df['Text'] = df['Text'].apply(remove_urls)

In [12]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"::My dear Heresiarchess, - Me being the ""mai...",0,2
2,:Warriorboy85 has clarified that he did not ...,0,3
3,"== English == Is ""shootdown"" a proper word?...",0,4
4,"} | width=""20%"" valign=""top"" | Archives __T...",0,5
...,...,...,...
94263,", like nonconvex uniform polyhedron",0,94264
94264,Niggers and their liberal friends steal everyt...,1,94265
94265,Not into dressin' to impress these bitches or ...,1,94266
94266,:Replacement ref - 1UP archive not working? ...,0,94267


In [13]:
(df[df['Text'].str.contains('"URL HERE"')])

,Text,label,id
61,":Look at this, everybody is Celtic probably ""...",0,62
67,RT @theblaze: Before CNN refrained from showin...,0,68
159,== Los_Angeles_Pollution.jpg file in Rachel ...,0,160
175,::I suspected as much. But this object has a ...,0,176
206,:Looking at articles on other VNSAs (Violent ...,0,207
...,...,...,...
94039,RT @ColeMcDowell2: I'm not sexist or anything ...,1,94040
94164,@harun_yahya Yahya flips out about PKK being c...,1,94165
94191,"RT @TRobinsonNewEra: ""URL HERE"" 8-year-old Yem...",1,94192
94201,"To be fair, if you take a look at the talk pa...",0,94202


In [14]:
df['Text'] = df['Text'].str.replace('=', '')

In [15]:
df[df['Text'].str.contains('=')]

,Text,label,id


In [16]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"::My dear Heresiarchess, - Me being the ""mai...",0,2
2,:Warriorboy85 has clarified that he did not ...,0,3
3,"English Is ""shootdown"" a proper word? ...",0,4
4,"} | width""20%"" valign""top"" | Archives __TOC...",0,5
...,...,...,...
94263,", like nonconvex uniform polyhedron",0,94264
94264,Niggers and their liberal friends steal everyt...,1,94265
94265,Not into dressin' to impress these bitches or ...,1,94266
94266,:Replacement ref - 1UP archive not working? ...,0,94267


In [17]:
df[df['Text'].str.contains('(UTC)')]

,Text,label,id
224,", 10 June 2009 (UTC) :::Thank you very much! ...",0,225
317,September 2013 (UTC) :::::The lines are back....,0,318
324,should be changed at all. The current wording ...,1,325
432,"(UTC) ::I agree. 11:18, 13 April 2012",0,433
485,2014 (UTC) ::So you like to review articles f...,0,486
...,...,...,...
93908,New Page Triage newsletter Hey all! Thanks ...,0,93909
93916,Tbma I think its very unfair to compare my...,0,93917
93962,January 2008 (UTC) Picture this: Wikipedia u...,0,93963
94024,April 2012 (UTC) Hi Borakai. It seems to me ...,0,94025


In [18]:
# regex patterns for date and time
date_regex1 = r"\d{4}\s[a-zA-Z]{3}\s\d{2}" #yyyy mmm dd
date_regex2 = r"\d{4}\s[a-zA-Z]{3}\s\d{1}" #yyyy mmm d
date_regex3 = r"\d{2}\s(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}" # dd month yyyy
date_regex4 = r"\d{2}\,\s(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}" # dd, month yyyy
date_regex5 = r"(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}" # month yyyy
date_regex6 = r"(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{2}\s\d{4}" # month dd yyyy
date_regex7 = r"(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{2}\,\s\d{4}" # month dd, yyyy
date_regex8 = r"(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1}\s\d{4}" # month d yyyy
date_regex9 = r"(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1}\,\s\d{4}" # month d, yyyy
date_regex10 = r"\d{2}\s[a-zA-z]{3}\,\s\d{4}" #dd mmm, yyyy
date_regex11 = r"\d{1}\s[a-zA-z]{3}\,\s\d{4}" #d mmm, yyyy
date_regex12 = r"[a-zA-Z]{3}\s\d{2}\,\s\d{4}" #mmm dd, yyyy
date_regex13 = r"[a-zA-Z]{3}\s\d{1}\,\s\d{4}" #mmm d, yyyy
time_regex1 = r"\d{2}\:\d{2}\,\s\d{2}"
time_regex2 = r"\d{2}\:\d{2}"

In [19]:
all_regex_patterns = [date_regex1, date_regex2, date_regex3, date_regex4, date_regex5,
                      date_regex6, date_regex7, date_regex8, date_regex9, date_regex10,
                      date_regex11, date_regex12, date_regex13, time_regex1, time_regex2]

In [20]:
for pattern in all_regex_patterns:
    df.loc[df['Text'].str.contains('UTC'), 'Text'] = df.loc[df['Text'].str.contains('UTC'), 'Text'].str.replace(pattern, '', regex=True)

In [21]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"::My dear Heresiarchess, - Me being the ""mai...",0,2
2,:Warriorboy85 has clarified that he did not ...,0,3
3,"English Is ""shootdown"" a proper word? ...",0,4
4,"} | width""20%"" valign""top"" | Archives __TOC...",0,5
...,...,...,...
94263,", like nonconvex uniform polyhedron",0,94264
94264,Niggers and their liberal friends steal everyt...,1,94265
94265,Not into dressin' to impress these bitches or ...,1,94266
94266,:Replacement ref - 1UP archive not working? ...,0,94267


In [22]:
df['Text'] = df['Text'].str.replace('UTC', '')

In [23]:
parentheses_regex = r"\(\)"
df['Text'] = df['Text'].str.replace(parentheses_regex, '', regex=True)

In [24]:
parentheses_regex = r"\(.*?\)"
df['Text'] = df['Text'].str.replace(parentheses_regex, '', regex=True)

In [25]:
df['Text'] = df['Text'].str.replace('~', '')

In [26]:
df['Text'] = df['Text'].str.replace('"URL HERE"', '')

In [27]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"::My dear Heresiarchess, - Me being the ""mai...",0,2
2,:Warriorboy85 has clarified that he did not ...,0,3
3,"English Is ""shootdown"" a proper word? ...",0,4
4,"} | width""20%"" valign""top"" | Archives __TOC...",0,5
...,...,...,...
94263,", like nonconvex uniform polyhedron",0,94264
94264,Niggers and their liberal friends steal everyt...,1,94265
94265,Not into dressin' to impress these bitches or ...,1,94266
94266,:Replacement ref - 1UP archive not working? ...,0,94267


In [28]:
df['Text'] = df['Text'].str.replace(':', '')

In [29]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"My dear Heresiarchess, - Me being the ""main ...",0,2
2,Warriorboy85 has clarified that he did not i...,0,3
3,"English Is ""shootdown"" a proper word? ...",0,4
4,"} | width""20%"" valign""top"" | Archives __TOC...",0,5
...,...,...,...
94263,", like nonconvex uniform polyhedron",0,94264
94264,Niggers and their liberal friends steal everyt...,1,94265
94265,Not into dressin' to impress these bitches or ...,1,94266
94266,Replacement ref - 1UP archive not working? ...,0,94267


In [30]:
df.drop_duplicates(subset=['Text'], inplace=True)

In [31]:
num_nan_rows = df.isna().any(axis=1).sum()
print(num_nan_rows)

0


In [32]:
df.shape

(94077, 3)

In [33]:
df = df.set_index(pd.Index(range(1, len(df)+1)))
df['id'] = df.index

In [34]:
df = df.reset_index(drop=True)

In [35]:
len(df[df['label'] == 0])

46930

In [36]:
len(df[df['label'] == 1])

47147

In [37]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"My dear Heresiarchess, - Me being the ""main ...",0,2
2,Warriorboy85 has clarified that he did not i...,0,3
3,"English Is ""shootdown"" a proper word? ...",0,4
4,"} | width""20%"" valign""top"" | Archives __TOC...",0,5
...,...,...,...
94072,", like nonconvex uniform polyhedron",0,94073
94073,Niggers and their liberal friends steal everyt...,1,94074
94074,Not into dressin' to impress these bitches or ...,1,94075
94075,Replacement ref - 1UP archive not working? ...,0,94076


In [38]:
df['Text'] = df['Text'].str.replace('@', ' @')

In [39]:
df['Text'] = df['Text'].str.replace('#', ' #')

In [40]:
#Code to remove @
df['Text'] = df['Text'].apply(lambda x : ' '.join([Text for Text in x.split()if not Text.startswith("@")]))

In [41]:
#Code to remove @
df['Text'] = df['Text'].apply(lambda x : ' '.join([Text for Text in x.split()if not Text.startswith("#")]))

In [42]:
df['Text'] = df['Text'].str.replace('RT', '')

In [43]:
df['Text'] = df['Text'].str.replace('MKR', '')

In [44]:
df['Text'] = df['Text'].str.replace('mkr', '')

In [45]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"My dear Heresiarchess, - Me being the ""main ca...",0,2
2,Warriorboy85 has clarified that he did not int...,0,3
3,"English Is ""shootdown"" a proper word? Better q...",0,4
4,"} | width""20%"" valign""top"" | Archives __TOC__ |",0,5
...,...,...,...
94072,", like nonconvex uniform polyhedron",0,94073
94073,Niggers and their liberal friends steal everyt...,1,94074
94074,Not into dressin' to impress these bitches or ...,1,94075
94075,Replacement ref - 1UP archive not working? |,0,94076


In [47]:
def remove_unicode(text):
    if isinstance(text, str):
        return text.encode('ascii', 'ignore').decode('utf-8')
    else:
        return text

df['Text'] = df['Text'].apply(remove_unicode)

In [48]:
df

,Text,label,id
0,nah he pussy he won't,1,1
1,"My dear Heresiarchess, - Me being the ""main ca...",0,2
2,Warriorboy85 has clarified that he did not int...,0,3
3,"English Is ""shootdown"" a proper word? Better q...",0,4
4,"} | width""20%"" valign""top"" | Archives __TOC__ |",0,5
...,...,...,...
94072,", like nonconvex uniform polyhedron",0,94073
94073,Niggers and their liberal friends steal everyt...,1,94074
94074,Not into dressin' to impress these bitches or ...,1,94075
94075,Replacement ref - 1UP archive not working? |,0,94076


In [49]:
df.to_csv('/content/drive/MyDrive/HATE_SPEECH_DETECTION_PROJECT/DATASET/Clean_dataset_for_EMBEDDING.csv')
